In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
entity_df = pd.read_pickle("../data/entity_context.pkl")
document_df = pd.read_pickle("../data/documents.pkl")

In [3]:
joined_df = entity_df.merge(document_df, on='Document', how='left')

In [4]:
df = pd.DataFrame()

In [5]:
#one-hot encoding of features
#document and entity numbers
df["Document"] = joined_df.Document
df["Entity"] = joined_df.Entity
#entity types
df["LOC"] = [1 if t == "LOC" else 0 for t in joined_df.Type]
df["ORG"] = [1 if t == "ORG" else 0 for t in joined_df.Type]
df["PER"] = [1 if t == "PER" else 0 for t in joined_df.Type]
#occurrences
df ["Occurrence"] = joined_df.Occurrence
#document lengths
df["Doc_length"] = joined_df.Length
#sources
df["24ur"] = [1 if source == "24ur" else 0 for source in joined_df.Source]
df["dnevnik"] = [1 if source == "dnevnik" else 0 for source in joined_df.Source]
df["finance"] = [1 if source == "finance" else 0 for source in joined_df.Source]
df["rtvslo"] = [1 if source == "rtvslo" else 0 for source in joined_df.Source]
df["zurnal24"] = [1 if source == "zurnal24" else 0 for source in joined_df.Source]
#document sentiments
df["Avg_sentiment"] = joined_df.Avg_sentiment
df["Sd_sentiment"] = joined_df.Sd_sentiment
df["Doc_neutral"] = [1 if sent == "neutral" else 0 for sent in joined_df.Sentiment_y]
df["Doc_negative"] = [1 if sent == "negative" else 0 for sent in joined_df.Sentiment_y]
df["Doc_positive"] = [1 if sent == "positive" else 0 for sent in joined_df.Sentiment_y]

In [6]:
with open("../data/negative_words_Kadunc.txt", "r", encoding="utf-8") as file:
    data = file.read()
    negative_words = data.split("\n")
    
with open("../data/positive_words_Kadunc.txt", "r", encoding="utf-8") as file:
    data = file.read()
    positive_words = data.split("\n")

In [7]:
adjectives = []
adverbs = []
positive = []
negative = []
for row in joined_df.Context:
    adj = 0
    adv = 0
    pos = 0
    neg = 0
    for tup in row:
        if tup[0] in positive_words:
            pos += 1
        if tup[0] in negative_words:
            neg += 1
        if tup[1] == "ADJ":
            adj += 1
        if tup[1] == "ADV":
            adv += 1
            
    adjectives.append(adj)
    adverbs.append(adv)
    positive.append(pos)
    negative.append(neg)

In [8]:
df["Context_adjectives"] = adjectives
df["Context_adverbs"] = adverbs
df["Context_positive"] = positive
df["Context_negative"] = negative
df["Sentiment"] = joined_df.Sentiment_x

In [9]:
df = df[df["Sentiment"] != 0]
df.reset_index(drop=True, inplace=True)
df.to_pickle("../data/features.pkl")

In [10]:
#unpickled_df = pd.read_pickle("../data/features.pkl")
#unpickled_df

,Document,Entity,LOC,ORG,PER,Occurrence,Doc_length,24ur,dnevnik,finance,...,Avg_sentiment,Sd_sentiment,Doc_neutral,Doc_negative,Doc_positive,Context_adjectives,Context_adverbs,Context_positive,Context_negative,Sentiment
0,1,1,0,0,1,11,2939,1,0,0,...,3.5,0.707,1,0,0,1,0,0,0,4
1,1,2,1,0,0,1,2939,1,0,0,...,3.5,0.707,1,0,0,0,0,0,0,3
2,1,3,0,1,0,6,2939,1,0,0,...,3.5,0.707,1,0,0,0,0,0,0,3
3,1,4,1,0,0,4,2939,1,0,0,...,3.5,0.707,1,0,0,0,0,0,0,3
4,1,5,0,1,0,5,2939,1,0,0,...,3.5,0.707,1,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14266,9966,25,1,0,0,1,2273,0,0,0,...,3,0,1,0,0,0,0,0,0,3
14267,9966,26,1,0,0,1,2273,0,0,0,...,3,0,1,0,0,0,0,0,0,3
14268,9966,27,1,0,0,1,2273,0,0,0,...,3,0,1,0,0,0,0,0,0,3
14269,9966,28,1,0,0,1,2273,0,0,0,...,3,0,1,0,0,0,0,0,0,3
